<a href="https://colab.research.google.com/github/sk511/Language_translator/blob/main/Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download datasets for training the model
# http://www.manythings.org/anki/

In [ ]:
#In this Translation, we will be using LSTM for translating

In [ ]:
from tensorflow.keras.models import Model
from  tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [ ]:
# initializing the required variables
batch_size=64     #batch size for training
epochs=100        #number of epoch to train for
latent_dim=256    #latent dimensionality of the encoding space
num_samples=10000 #Number of samples to train on
data_path=r'/content/hin_eng.txt'      #path to the dataset

In [ ]:
# vectorize the data(converting the text into vectors)
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path,'r',encoding='utf-8') as f:
  lines=f.read().split('\n')
for line in lines[:min(num_samples,len(lines)-1)]:
  input_text,target_text,_=line.split('\t')
  target_text='\t'+target_text+'\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [ ]:
len(target_characters)

90

In [ ]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [ ]:
print('Number of samples: ',len(input_texts))
print('Number of unique input tokens: ',num_encoder_tokens)
print('Number of unique output tokens: ',num_decoder_tokens)
print('Max sequence length for inputs: ',max_encoder_seq_length)
print('Max sequence length for outputs: ',max_decoder_seq_length)

Number of samples:  2979
Number of unique input tokens:  70
Number of unique output tokens:  90
Max sequence length for inputs:  107
Max sequence length for outputs:  123


In [ ]:
input_token_index=dict(
    [(char,i) for i, char in enumerate(input_characters)])
target_token_index=dict(
    [(char,i) for i, char in enumerate(target_characters)])

In [ ]:
target_token_index

In [ ]:
encoder_input_data=np.zeros(
    (len(input_texts),max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data=np.zeros(
    (len(input_texts),max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data=np.zeros(
    (len(input_texts),max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [ ]:
encoder_input_data

In [ ]:
# one hot representation
for i, (input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]]=1
  encoder_input_data[i,t+1:,input_token_index[' ']]=1
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[char]]=1
    if t>0:
      decoder_target_data[i,t-1,target_token_index[char]]=1
  decoder_input_data[i,t+1:,target_token_index[' ']]=1
  decoder_target_data[i,t:,target_token_index[' ']]=1

In [ ]:
encoder_input_data[0].shape

(107, 70)

In [ ]:
# Define an input sequence and process it
encoder_inputs=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs, state_h, state_c=encoder(encoder_inputs)
# we discard the encoder_output and only keep the states
encoder_states=[state_h,state_c]

In [ ]:
# set up the decoder using encoder_states as initial states
decoder_inputs=Input(shape=(None,num_decoder_tokens))
# we setup our decoder to return full output sequence and internal
# states. We don't use the return states in the training model
# but we will use them in inference.
decoder_lstm=LSTM(latent_dim, return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,
                                 initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens, activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [ ]:
# Defining the model that will turn encoder and decoder input data
# into decoder target data
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)

# Training the model
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
38/38 [==============================] - 55s 1s/step - loss: 1.2870 - accuracy: 0.7853 - val_loss: 1.6085 - val_accuracy: 0.6869
Epoch 2/100
38/38 [==============================] - 50s 1s/step - loss: 1.0098 - accuracy: 0.8083 - val_loss: 1.6662 - val_accuracy: 0.6844
Epoch 3/100
38/38 [==============================] - 50s 1s/step - loss: 0.8897 - accuracy: 0.8080 - val_loss: 1.4936 - val_accuracy: 0.6873
Epoch 4/100
38/38 [==============================] - 51s 1s/step - loss: 0.8468 - accuracy: 0.8089 - val_loss: 1.4527 - val_accuracy: 0.6873
Epoch 5/100
38/38 [==============================] - 51s 1s/step - loss: 0.8769 - accuracy: 0.8026 - val_loss: 1.4200 - val_accuracy: 0.6873
Epoch 6/100
38/38 [==============================] - 54s 1s/step - loss: 0.8068 - accuracy: 0.8089 - val_loss: 1.3587 - val_accuracy: 0.6873
Epoch 7/100
38/38 [==============================] - 52s 1s/step - loss: 0.7933 - accuracy: 0.8083 - val_loss: 1.3138 - val_accuracy: 0.6872
Epoch 8/100
3

In [ ]:
# import pickle
# pickle.dump(model,open('fre_eng.pkl','wb'))

import tensorflow as tf

# Assume you have a TensorFlow model 'model'
model.save('/content/model')

In [ ]:
from tensorflow.keras.models import load_model

# Assume you have a Keras model 'model'
model.save('/content/model.h5')

In [ ]:
import tensorflow as tf

# Load the original TensorFlow model
loaded_model = tf.keras.models.load_model('/content/model')

# Convert the model to TensorFlow Lite format with Graph Mode serialization
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.experimental_enable_resource_variables = True
tflite_model = converter.convert()

# Save the converted model to a .tflite file
with open('converted_model.tflite', 'wb') as f:
    f.write(tflite_model)